# Pré-tratamento

Nesta secção irá ser feito o pré-tratamento dos dados. Concretamente iremos fundir os dados provenientes dos diferentes datasets, "alinhando-os" pelas colunas "Entidade" (equivalente a nome do país), "Code" (equivalente ao código identificador IUPAC do país) e "Year" (correspondente ao ano em que foram recolhidos os dados).

Iremos também "aparar" os dados em função do indicador limitante, neste caso o ano de dados disponíveis, sendo o período de 1990 a 2016 o período comum a todos os datasets.

Para efeitos de comparação iremos fazer merge dos datasets usando o package Pandas e Spark, de modo a comparar a eficiência.

### Pré-tratamento utilizando Pandas

In [94]:
# Com pandas
import pandas as pd

# Importamos o módulo time para efeitos de comparação de tempos de execução
import time

pd_start = time.time()

# Carregamos o dataset com dados populacionais
import pandas as pd
pop = pd.read_csv('population.csv')
pop = pop[pop['Year'] >= 1990]
pop = pop[pop['Year'] <= 2016]

# Carregamos o dataset para prevelência de obesidade
obes = pd.read_csv('share-of-adults-defined-as-obese.csv')
obes = obes[obes['Year'] >= 1990]

# Carregamos o dataset para prevalência doenças mentais
mental = pd.read_csv('mental-illnesses-prevalence.csv')
mental = mental[mental['Year'] <= 2016]

merged = pd.DataFrame()
merged = pd.merge(pop,obes,on=['Entity','Code','Year'])
merged = pd.merge(merged,mental,on=['Entity','Code','Year'])



pd_end = time.time()

pd_elapsed = pd_end - pd_start

merged.head()

,Entity,Code,Year,Population (historical estimates),"Indicator:Prevalence of obesity among adults, BMI &GreaterEqual; 30 (crude estimate) (%) - Sex:Both sexes",Schizophrenia disorders (share of population) - Sex: Both - Age: Age-standardized,Depressive disorders (share of population) - Sex: Both - Age: Age-standardized,Anxiety disorders (share of population) - Sex: Both - Age: Age-standardized,Bipolar disorders (share of population) - Sex: Both - Age: Age-standardized,Eating disorders (share of population) - Sex: Both - Age: Age-standardized
0,Afghanistan,AFG,1990,10694804,1.0,0.223206,4.996118,4.713314,0.703023,0.127700
1,Afghanistan,AFG,1991,10745168,1.1,0.222454,4.989290,4.702100,0.702069,0.123256
2,Afghanistan,AFG,1992,12057436,1.2,0.221751,4.981346,4.683743,0.700792,0.118844
3,Afghanistan,AFG,1993,14003764,1.2,0.220987,4.976958,4.673549,0.700087,0.115089
4,Afghanistan,AFG,1994,15455560,1.3,0.220183,4.977782,4.670810,0.699898,0.111815


### Carregamento e pré-tratamento de dados utilizando Spark (package PySpark)

In [98]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

import time

spark_start = time.time()

# Criação de uma sessão Spark
Spark = SparkSession.builder.appName("BigDataAnalysis").getOrCreate()

# Carregar e aparar datasets

# Dataset da população
spark_pop = Spark.read.csv(
    'population.csv',
    header = True ,
    inferSchema = True
    )

spark_pop = spark_pop.filter(col('Year').between(1990, 2016))

# Carregamos o dataset para prevelência de obesidade
spark_obes = Spark.read.csv(
    'share-of-adults-defined-as-obese.csv',
    header = True,
    inferSchema = True
    )

spark_obes = spark_obes.filter(col('Year') >= 1990)

# Carregamos o dataset para prevalência doenças mentais
spark_mental = Spark.read.csv(
    'mental-illnesses-prevalence.csv',
    header = True,
    inferSchema = True
    )

spark_mental = spark_mental.filter(col('Year') <= 2016)

# Unimos os datasets com o mesmo critério (país, IUPAC, código)

spark_merged = spark_pop.join \
  (spark_obes, on = ['Entity','Code','Year'], how = 'inner').join \
  (spark_mental, on = ['Entity', 'Code','Year'], how = 'inner')

# Descomentar para ver o dataset
#spark_merged.show()

Spark.stop()

spark_end = time.time()

spark_elapsed = spark_end - spark_start

In [99]:
# Comparamos o tempo de computação para cada um dos modelos

print(f'Com pandas, demorou {round(pd_elapsed, 3)} segundos',
      f'Com spark, demorou {round(spark_elapsed, 3)} segundos',
      sep = '\n')

Com pandas, demorou 0.076 segundos
Com spark, demorou 2.111 segundos


Neste caso concreto percebemos que várias ferramentas têm comportamentos diferentes em função dos dados, o que é algo a ter em conta no contexto do que pretendemos fazer.

Numa próxima abordagem iremos perceber que em função do que podemos fazer há ferramentas/abordagens mais ou menos eficientes. Neste caso concreto, a utilização de Spark demonstrou-se como menos eficiente.